# BLMBISA!!

# Building label using code

## check n run dataset

In [1]:
import pandas as pd

In [2]:
test_df  = pd.read_csv('../The/test.csv')
train_df = pd.read_csv('../The/train.csv')
train_unlabelled_df = pd.read_csv('../The/train_unlabelled.csv', index_col = [0])

In [3]:
test_df.head()

,id,text,Label
0,1316740974190630126,titisan air mata kini kekeringan,0
1,1316740974190631210,tni polri serta satpol pp dki menyambangi loka...,0
2,1355811699857604608,"Gempa Mag:5.2, 31-Jan-21 16:28:55 WIB, Lok:5....",1
3,1316740974190629802,berbagai klaster penyebaran covid 19 kita moni...,0
4,1316740974190629578,saya tetap optimistis perekonomian indonesia a...,0


In [4]:
train_df.head()

,id,text,Label
0,1346427759858393089,"Gempa Mag:4.8, 05/01/2021 18:56:21 (Pusat gem...",1
1,1316740974190631449,idha nama pemilik perusahaan yg bikin banjir a...,0
2,1316740974190630394,web kritik film danieldokter tariz aryaapepe r...,0
3,1316740974190630928,penjelasan fpi ganti nama jadi front persatuan...,0
4,1320644218503581696,"Gempa Mag:5.0, 26-Oct-20 15:26:37 WIB, Lok:0....",1


In [20]:
train_df.count()

id       4234
text     4234
Label    4234
dtype: int64

In [5]:
train_unlabelled_df.head()

,text
0,"Gempa Mag:4.8, 05/01/2021 18:56:21 (Pusat gem..."
1,idha nama pemilik perusahaan yg bikin banjir a...
2,web kritik film danieldokter tariz aryaapepe r...
3,penjelasan fpi ganti nama jadi front persatuan...
4,"Gempa Mag:5.0, 26-Oct-20 15:26:37 WIB, Lok:0...."


## Import Dataset to Rubrix

In [6]:
# run python -m rubrix on CMD 1st

In [1]:
import rubrix as rb

ModuleNotFoundError: No module named 'rubrix'

In [8]:

# build records from the train dataset
records = [
    rb.TextClassificationRecord(
        text=row.text,
        #metadata={"id": row.id} 
    )
    for i,row in train_unlabelled_df.iterrows()
]

# build records from the test dataset with annotation
labels = ["NOT_ND", "ND"]
records += [
    rb.TextClassificationRecord(
        text=row.text,
        annotation=labels[row.Label],
        #metadata={"id": row.id}
    )
    for i,row in test_df.iterrows()
]

# log records to Rubrix
rb.log(records, name="ws_nd_codelablled")


C:\Users\arkan\anaconda3\envs\Skripshit\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|█████████████████████████████████████████████████████████████████████████████| 6049/6049 [00:40<00:00, 149.35it/s]

6049 records logged to http://localhost:6900/datasets/rubrix/ws_nd_codelablled


BulkResponse(dataset='ws_nd_codelablled', processed=6049, failed=0)

## Label making

In [9]:
from rubrix.labeling.text_classification import Rule, WeakLabels

In [29]:
def rule_1(record: rb.TextClassificationRecord):
    for word in "bmkg|wib|lok|mmi|dirasakan|laut|berada|magnitudo".split("|"):
      if word in record.inputs["text"]:
        return "ND"
      else:
        return "NOT_ND"

def rule_2(record: rb.TextClassificationRecord):
    for word in "untuk|ada|itu|saya|kita|bisa|ngga|gimana".split("|"):
      if word in record.inputs["text"]:
        return "NOT_ND"

## Label Eval

In [30]:
from rubrix.labeling.text_classification import load_rules

rules = [rule_1, rule_2]
rules += load_rules(dataset="ws_nd_codelablled")

weak_label_code = WeakLabels(
    rules=rules,
    dataset="ws_nd_codelablled"
)

weak_label_code.summary()

Applying rules: 100%|██████████████████████████████████████████████████████████| 6049/6049 [00:00<00:00, 241719.34it/s]


,label,coverage,annotated_coverage,overlaps,conflicts,correct,incorrect,precision
rule_1,"{ND, NOT_ND}",1.000000,1.000000,0.632005,0.005621,878,937,0.483747
rule_2,{NOT_ND},0.632005,0.627548,0.632005,0.005621,543,596,0.476734
total,"{ND, NOT_ND}",1.000000,1.000000,0.632005,0.005621,1421,1533,0.481043


In [31]:
from rubrix.labeling.text_classification import MajorityVoter

# instantiate the majority vote label model by simply providing the weak labels object
majority_model = MajorityVoter(weak_label_code)

In [32]:
# check its performance
print(majority_model.score(output_str=True))

              precision    recall  f1-score   support

          ND       0.00      0.00      0.00       926
      NOT_ND       0.49      0.99      0.65       882

    accuracy                           0.49      1808
   macro avg       0.24      0.50      0.33      1808
weighted avg       0.24      0.49      0.32      1808



In [26]:
records = majority_model.predict()
label2int = {'NOT_ND': 0, 'ND': 1}

In [27]:
training_data = pd.DataFrame(
    [
        {"text": rec.text, "label": label2int[rec.prediction[0][0]]}
        for rec in records
    ]
)

In [34]:
training_data

,text,label
0,"Gempa Dirasakan Magnitudo: 5.1, Kedalaman: 36...",0
1,"Gempa Magnitudo: 5.1, Kedalaman: 10 km, 25 Me...",0
2,saya juga meminta agar ketimpangan kapasitas t...,0
3,coba lu bayangin kiamat 2012 itu typo jadi 202...,0
4,kan bumi disitu kacaunya sama bencana alam jug...,0
...,...,...
4202,soal pelarangan fpi sosiolog pemerintah perlu ...,0
4203,jangan kau tutup indahmu dengan lahar panasmu ...,0
4204,Tiga kecamatan di wilayah Sukabumi terdampak b...,0
4205,kmhdi galang bantuan untuk korban banjir dan g...,0


In [35]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4207 entries, 0 to 4206
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    4207 non-null   object
 1   label   4207 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 65.9+ KB


In [36]:
training_data.query('label == 1')

,text,label
84,soal dentuman ini bagiannya pvmbg jadi bukan b...,1
226,jadi bmkg ga boleh posting gempa vulkanik,1
260,dentuman yang terdengar di jabodetabek tengah ...,1
329,ini kalau disusun kalimat dentuman dajjal krak...,1
685,maap y bmkg tu utuk gempa tsunami ini gunung y...,1
727,iyaa beb gempa lumayan katanya bmkg jg udh upd...,1
1228,dan aplikasi bmkg g menampakkan sesuatu apapun...,1
1490,yaallah pagi2 trending aq serem amat si krakat...,1
1921,btw kok bmkg ga ngomong apa dah cuma gempa yg ...,1
2239,indigo kiamat dajjal ya allah bunyi kedengeran...,1


In [37]:
training_data.groupby('label').nunique()

,text
label,
0,4156
1,16
